###  Jupyter notebook for feature demonstration of `metabolinks` Python module

#### Metabolinks Github home: https://github.com/aeferreira/metabolinks

Install metabolinks by running

`pip install metabolinks`

## Peak alignment for peak tables contained in MS-Excel sheets

This notebook demonstrates peak alignment, based on m/z proximity.

A given **m/z ppm tolerance** defines the maximum relative deviation in the values of m/z from different peak lists to be considered the same peak in the resulting aligned peak table.

Each peak list is a table of |m/z , Signal| values. Intensities are not considered, just copied over to the final table, and m/z values are averaged in each aligned group.

Sample names are also copied over to the final table. This notebook illustrates the assignment of labels to data

An alignment is performed within each Excel worksheet. This means that several peak tables should be contained in the same worksheet. In the example data, in file `sample_data.xlsx`, each excel worksheet contains three samples obtained under different sample extraction methods.

This notebook also illustrates saving the results to an Excel file and to a CSV file.

### Function `align()`

The function `align()` performs the correspondence between peak lists and joins the data in a new table.

Parameters of this function are:

- a list (or any sequence) of Pandas DataFrames with data organized with "samples in columns". The index of the DataFrame should be interpreted as m/ values.

- `ppmtol` the tolerance, in ppm, for the deviation between m/z values of the same peak in different tables. Default is 1 ppm.
- `min_samples` reproducibility threshold: the minimum number of occurances of a given peak to be included in the aligned table. Default is 1.

- `return_alignment_desc` (default is False) whether to return stats for each group found.

Returns a Pandas DataFrame with the peak lists aligned.

### Read in data

In [ ]:
import pandas as pd
from metabolinks import add_labels, read_data_from_xcel, align

file_name = 'sample_data.xlsx'
out_fname = 'aligned_data.xlsx'

data_sets = read_data_from_xcel(file_name, header=[0, 1], drop_header_levels=1)

data_sets

### Perform alignment

In [ ]:
ppmtol = 1.0
min_samples = 1
labels = ['wt', 'mod', 'mod']

print(f'------ Aligning tables in each Excel sheet {file_name} ...')
results = {}
for d in data_sets:
    print(f'\n+++++++ sheet {d} +++++++')
    aligned, desc = align(data_sets[d],
                          min_samples=min_samples, 
                          ppmtol=ppmtol,
                          return_alignment_desc=True,
                          verbose=True)
    
    aligned = add_labels(aligned, labels)
    aligned.columns.names = ['label', 'sample']
    
    print('\n--- Result: --------------------')
    print(aligned)
    print('+++++++++++++++++++++++++++++')
    # keep results in a dictionary
    results[d] = aligned
    results['groups {}'.format(d)] = desc


### Save aligned sheets to Excel

In [ ]:
with pd.ExcelWriter(out_fname) as writer:
    for sname in results:
        results[sname].to_excel(writer, sheet_name=sname)


### Save results to CSV (several files, one per sheet)

In [ ]:
for sname in results:
    if not sname.startswith('groups'): # skip descriptions of groups
        name = f'aligned_{sname}.csv'
        results[sname].to_csv(name)